In [20]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()
# Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))
# Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Flattening
classifier.add(Flatten())

# Full connection
classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(Dense(output_dim = 3, activation = 'softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

C:\anaconda\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3...)`
  # Remove the CWD from sys.path while we load stuff.
C:\anaconda\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
C:\anaconda\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
C:\anaconda\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=3)`


In [24]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

checkpointer = ModelCheckpoint(filepath='modelProject.hdf5', 
                               verbose=1, save_best_only=True)

classifier.fit_generator(training_set,
                         samples_per_epoch = 4000,
                         nb_epoch = 10,
                         validation_data = test_set,
                         nb_val_samples = 1000,
                        callbacks = [checkpointer])

Found 63 images belonging to 3 classes.
Found 32 images belonging to 3 classes.


C:\anaconda\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:29: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\anaconda\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=125, validation_data=<keras_pre..., callbacks=[<keras.ca..., epochs=10, validation_steps=1000)`


Epoch 1/10
125/125 [==============================] - 526s 4s/step - loss: 0.0254 - acc: 0.9949 - val_loss: 3.3591 - val_acc: 0.4375

Epoch 00001: val_loss improved from inf to 3.35912, saving model to modelProject.hdf5
Epoch 2/10
125/125 [==============================] - 526s 4s/step - loss: 0.0040 - acc: 1.0000 - val_loss: 3.8419 - val_acc: 0.4375

Epoch 00002: val_loss did not improve from 3.35912
Epoch 3/10
125/125 [==============================] - 525s 4s/step - loss: 0.0042 - acc: 0.9997 - val_loss: 4.2486 - val_acc: 0.3438

Epoch 00003: val_loss did not improve from 3.35912
Epoch 4/10
125/125 [==============================] - 492s 4s/step - loss: 0.0013 - acc: 1.0000 - val_loss: 3.8742 - val_acc: 0.4375

Epoch 00004: val_loss did not improve from 3.35912
Epoch 5/10
125/125 [==============================] - 501s 4s/step - loss: 6.4367e-04 - acc: 1.0000 - val_loss: 4.2578 - val_acc: 0.4062

Epoch 00005: val_loss did not improve from 3.35912
Epoch 6/10
125/125 [================

In [65]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('random8.png', target_size =(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
print(training_set.class_indices)
print(result)

{'nudges': 0, 'unpleasants': 2, 'persuasives': 1}
[[0. 1. 0.]]


In [63]:
# serialize model to JSON
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
